In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px
import plotly.graph_objects as go
import importlib
import performance_shr
importlib.reload(performance_shr)
from performance_shr import convert_to_float

In [ ]:
default_threshold = 0.10   # eg 0.1 = 10%
file_dir = 'D:/Astro/Images/2025-10-03/String of Pearls/LIGHT'
file_name = 'ImageMetaData.csv'
file_path = os.path.join(file_dir, file_name)
converters_dict = {
    'FilePath': str,   
    'Gain': convert_to_float,   
    'DetectedStars': convert_to_float,   
    'HFR': convert_to_float,  
    'HFRStDev': convert_to_float,  
    'FWHM': convert_to_float,  
    'Eccentricity': convert_to_float,  
    'FocuserPosition': convert_to_float,  
    'Airmass': convert_to_float,  
    'MountRA': convert_to_float,  
    'MountDec': convert_to_float,  
    }

In [ ]:
data = pd.read_csv(file_path, converters=converters_dict)
data['FileName'] = data['FilePath'].apply(lambda x: os.path.basename(x))
data = data[data['FilePath'].str.strip() != '' ]
data

In [ ]:
def filter_percentiles(df, col='HFR', topq=False, bottomq=False, top=False, bottom=False, zero=True):
    top_threshold = top if top else df[col].quantile(1 - topq) if topq else False
    bottom_threshold = bottom if bottom else df[col].quantile(bottomq) if bottomq else False
    is_na = df[col].isna()
    is_false = is_na != is_na
    is_zero = df[col] == 0 if zero else is_false
    is_top = df[col] >= top_threshold if top_threshold else is_false
    is_bottom = df[col] <= bottom_threshold if bottom_threshold else is_false
    is_filtered = is_zero | is_na | is_top | is_bottom

    fig=go.Figure()
    color_map = {True: 'red', False: 'blue'}
    fig.add_trace(go.Scatter(x=df.ExposureNumber, y=df[col], name=col, mode='markers', marker=dict(color=np.vectorize(color_map.get)(is_filtered))))
    fig.update_layout(title=f'{col} vs Exposure No.', xaxis_title='Exposure No.', yaxis_title=col)
    fig.show()    

    print(f'{col}: TopThreshold={top_threshold:.3f} n({is_top.sum()}), BottomThreshold={bottom_threshold:.3f} n({is_bottom.sum()}), na=n({is_na.sum()}), zero=n({is_zero.sum()}), filtered=n({is_filtered.sum()})\n\n')
    return is_filtered

In [ ]:
print(f'**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****')
is_badHFR = filter_percentiles(data, col='HFR', topq=0.1)
is_badHFRStDev = filter_percentiles(data, col='HFRStDev', topq=0.1, bottom=0.17)
is_badFWHM = filter_percentiles(data, col='FWHM', topq=.08, bottomq=0.01)
is_badEccentricity = filter_percentiles(data, col='Eccentricity', topq=0.1, bottomq=0.1)
is_badDetectedStars = filter_percentiles(data, col='DetectedStars', bottom=60)
is_rejected = is_badHFR | is_badHFRStDev | is_badFWHM |  is_badFWHM | is_badEccentricity | is_badDetectedStars
print(f'\nTotal Rejected: n({is_rejected.sum()}/{len(data)}, Total Images: n({len(data) - is_rejected.sum()})')


In [ ]:
# Create the 'rejected' directory at the same level as file_dir if it doesn't exist 
parent_dir = os.path.dirname(file_dir) 
rejected_dir = os.path.join(parent_dir, 'Rejected') 
os.makedirs(rejected_dir, exist_ok=True)

In [ ]:
rejects = data[is_rejected].FileName
for filename in rejects:
    src_path = os.path.join(file_dir, filename)
    dest_path = os.path.join(rejected_dir, filename)
    if os.path.exists(src_path): 
        shutil.move(src_path, dest_path) 
        print(f'Moved: {filename}')

print('File moving complete.')